In [ ]:
def radec_to_xyz(ra, dec):
    x = math.cos(np.deg2rad(dec)) * math.cos(np.deg2rad(ra))
    y = math.cos(np.deg2rad(dec)) * math.sin(np.deg2rad(ra))
    z = math.sin(np.deg2rad(dec))

    return np.array([x, y, z], dtype=np.float64)

#functions from sncosmo
def cmb_dz(ra, dec):
    """See http://arxiv.org/pdf/astro-ph/9609034
     CMBcoordsRA = 167.98750000 # J2000 Lineweaver
     CMBcoordsDEC = -7.22000000
    """

    # J2000 coords from NED
    CMB_DZ = 371000. / 299792458.
    CMB_RA = 168.01190437
    CMB_DEC = -6.98296811
    CMB_XYZ = radec_to_xyz(CMB_RA, CMB_DEC)

    coords_xyz = radec_to_xyz(ra, dec)
    
    dz = CMB_DZ * np.dot(CMB_XYZ, coords_xyz)

    return dz

def cmb_to_helio(z, ra, dec):
    """Convert from CMB-frame redshift to heliocentric redshift.
    
    Parameters
    ----------
    z : float
        CMB-frame redshift.
    ra, dec: float
        RA and Declination in degrees (J2000).
    """

    dz = -cmb_dz(ra, dec)
    one_plus_z_pec = math.sqrt((1. + dz) / (1. - dz))
    one_plus_z_helio = (1. + z) * one_plus_z_pec

    return one_plus_z_helio - 1.

def one_lenght_redshift(data):
    
    redshift_value = None
    
    #if there is kind in the list continue
    if "kind" in data["redshift"][0]:

        #if its heliocentric pick then
        if data["redshift"][0]["kind"] == "heliocentric":

            redshift_value = float(data["redshift"][0]["value"])
            print(redshift_value)
        
        #if its cmb pick then    
        if data["redshift"][0]["kind"] == "cmb":

            #convertion of hours to degrees
            if data["ra"][0]["u_value"] == "hours" and data["dec"][0]["u_value"] == "degrees":

                c = SkyCoord(str(data["ra"][0]["value"]),str(data["dec"][0]["value"]), unit=(units.hourangle, units.deg))

            elif data["ra"][0]["u_value"] == "hours" and data["dec"][0]["u_value"] == "hours":

                c = SkyCoord(str(data["ra"][0]["value"]),str(data["dec"][0]["value"]), unit=(units.hourangle, units.hourangle))

            else:

                print("Erro!")

            redshift_value = float(cmb_to_helio(float(data["redshift"][0]["value"]), c.ra.deg, c.dec.deg))
            print(redshift_value)
      
    #if there is only 1 and its not specificated
    else:
            
        redshift_value = float(data["redshift"][0]["value"])
        print(redshift_value)
    
    return redshift_value

def redshift_selection(data):
    
    redshift_value = None
    
    for nredshift in range(0,len(data["redshift"])):
            
        #if there is kind in the list continue
        if "kind" in data["redshift"][nredshift]:

            #if its heliocentric pick then
            if data["redshift"][nredshift]["kind"] == "heliocentric":

                redshift_value = float(data["redshift"][nredshift]["value"])
                print(redshift_value)
                break
                
            
            #if its cmb 
            if data["redshift"][nredshift]["kind"] == "cmb":

                #convertion of hours to degrees
                if data["ra"][nredshift]["u_value"] == "hours" and data["dec"][nredshift]["u_value"] == "degrees":

                    c = SkyCoord(str(data["ra"][nredshift]["value"]),str(data["dec"][nredshift]["value"]), unit=(units.hourangle, units.deg))

                elif data["ra"][nredshift]["u_value"] == "hours" and data["dec"][nredshift]["u_value"] == "hours":

                    c = SkyCoord(str(data["ra"][nredshift]["value"]),str(data["dec"][nredshift]["value"]), unit=(units.hourangle, units.hourangle))

                else:

                    print("Erro 1!")

                #convertion of cmb to heliocentric
                redshift_value = float(cmb_to_helio(float(data["redshift"][nredshift]["value"]), c.ra.deg, c.dec.deg))
                print(redshift_value)
                break
            
    return redshift_value


def redshift_HB(data):
    redshift_value = None
    
    maxredshift = None
    
    sumredshift = 0
    HB_test = 0
    for nredshift in range(0,len(data["redshift"])):
            
        #if not realize a sum to obtain a mean value
        if "kind" not in data["redshift"][nredshift]:
            
            sumredshift = sumredshift + float(data["redshift"][nredshift]["value"])
                
            #if the sn is in the hubnle flow realize a mean value of the redshifts
            if float(data["redshift"][nredshift]["value"]) > 0.15:
                
                HB_test = 1
                
            #if not the first redshift is the max redshift
            if nredshift == 0:
                maxredshift = float(data["redshift"][nredshift]["value"])
                
            if nredshift > 0 and maxredshift is not None:
                #then analise if the next redshift is grater than max redshift 
                if float(data["redshift"][nredshift]["value"]) > maxredshift:
                        
                    maxredshift = float(data["redshift"][nredshift]["value"])
        
    if HB_test == 1:
            
        #if sne is in the hubble flow realize a mean value
        redshift_value = sumredshift/len(data["redshift"])
        print(redshift_value)  
    elif HB_test == 0:
            
        #if not the maximum redshift is the heliocentric redshift
        redshift_value = maxredshift
        print(redshift_value)
        
    return redshift_value

def redshift_value(data):
    redshift_func = None
    #if there is 1 value use it
    if len(data["redshift"]) == 1:
        
        redshift_func = one_lenght_redshift(data)

    #if not...
    if len(data["redshift"]) == 1 and redshift_func is None:
        
        print("redshift nao encontrado")
        
    if len(data["redshift"]) > 1:
        
        redshift_func = redshift_selection(data)
                
    if len(data["redshift"]) > 1 and redshift_func is None:
        redshift_func = redshift_HB(data)
        
    return redshift_func

def max_time(data):
   
    t0dat = data["maxdate"][0]["value"]
    t0_max_photo = t0dat.split("/")
    t0_max_photo=str(t0_max_photo[0])+'-'+str(t0_max_photo[1])+'-'+str(t0_max_photo[2])
    t0_max_photo=Time(t0_max_photo).mjd
        
    return t0_max_photo
    
def suavization(x, freq):
    fc = 0.00833
    nf = fc/(freq/2)
    #print(nf)
    b, a = signal.butter(8, nf)
    y = signal.filtfilt(b, a, x, padlen=0)
    
    return y